# Erstellung eines Datensets 

Datensets können über die DNB auf verschiedenen Wegen bezogen oder selbst erstellt werden. Im Folgenden werden zwei Möglichkeiten der Erstellung und des Bezugs eigener Datensets beschrieben, die jeweils über die SRU-Schnittstelle der DNB erzeugt werden.

### Grundlagen zur SRU-Schnittstelle der DNB:

Die DNB bietet Ihre Daten auf drei verschiedene "Kataloge" gesplittet an, von denen immer einer für eine Abfrage ausgewählt werden muss. Dies geschieht über eine Erweiterung der Base-URL. Zur Verfügung stehen folgende Kataloge:

- Katalog der Deutschen Nationalbibliothek (DNB) - hierin befinden sich die Titeldaten
- Katalog des Deutschen Musikarchivs (DMA) - Datensätze des Deutschen Musikarchivs
- Katalog der Gemeinsamen Normdatei (GND) - hierin befinden sich die Normdaten

Die erweiterungen für die URL sind folgende:

- DNB: https://services.dnb.de/sru/dnb
- DMA: https://services.dnb.de/sru/dnb.dma
- GND: https://services.dnb.de/sru/authorities

Werden die jeweiligen Bereiche ohne weitere Spezifikationen abgefragt, senden sie eine "Explain-Response" in XML zurück. 

[Weitere Informationen und Dokumentation zur SRU-Schnittstelle](https://www.dnb.de/sru)


## 1. Erstellung eines Datensets mit Python

### Laden benötigter Zusatzbibliotheken: 

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


### SRU-Anfrage

In [ ]:
#Base-URL für den  Katalog der DNB (Titeldaten):
dnb = requests.get("https://services.dnb.de/sru/dnb")

#Einfache Amfrage an die Schnittstelle ohne Suche 
response = soup(dnb.content, features="xml")  #Parsen der Schnittstellenantwort als xml
print(response.prettify()[0:500])  #Einschränken der Ausgabe auf die ersten 500 Zeichen

Für eine Suchanfrage an die Daten der DNB wird nun zunächst über die Wahl der URL der Katalog definiert. Mit Hilfe der Variable parameter werdem dann alle weiteren Parameter, die die SRU-Schnittstelle benötigt, übergeben.

In [ ]:
#Base-URL:
dnb_url = "https://services.dnb.de/sru/dnb"

#Parameter, die mit der Anfrage übergeben werden sollen: 
parameter = {'version' : '1.1' , 
             'operation' : 'searchRetrieve' , 
             'query' : 'Klimawandel and jhr=2010', 
             'recordSchema' : 'MARC21-xml'} 

#Abfrage der Schnittstelle:
r = requests.get(dnb_url, params = parameter)
print(r.url)
#Parsen der Antwort in xml:
response = soup(r.content, features="xml")
#Überführen der einzelnen, in der Antwort enthaltenen Datensätze/Treffer ('Records') in eine Liste:
result = response.find_all('record', {'type':'Bibliographic'})
print(response.prettify()[0:750])

In [ ]:
number = response.find('numberOfRecords')
print(number.text, 'Ergebnisse')
print(len(result))

Wichtig: Die SRU-Schnittstelle liefert per default zunächst immer nur 10 Datensätze auf einmal aus, maximal 100.  

### Umwandlung in eine Funktion:

Funktion, die dank while-Schleife alle Datensätze der Anfrage sammelt:  

In [ ]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    i = 0 
    records = []

    while i % 100 == 0: 
        params.update({'startRecord': i})       
        r = requests.get(base_url, params=params)
        xml = soup(r.content, features="xml")
        result = xml.find_all('record', {'type':'Bibliographic'})
        records+=result
        num_results = len(result)
        if num_results != 100:
            break
        else: 
            i += 100  
        
    return records

In [ ]:
records = dnb_sru('Studentenbewegung and location=onlinefree and cod=pdf')
print(len(records), 'Ergebnisse')

In [ ]:
print(records[54].prettify())

Datenset als XML-Datei speichern: 

In [ ]:
with open ("datadump.xml", "w", encoding="utf-8") as f:
    f.write(f'''<?xml version="1.0" encoding="utf-8"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/"> 
                <version>1.1</version><numberOfRecords>{str(len(records))}</numberOfRecords>
                <records><record><recordSchema>MARC21-xml</recordSchema><recordPacking>xml</recordPacking><recordData>''')
    for record in records:
        f.write(str(record))
    f.write(f'''</recordData></record></records><echoedSearchRetrieveRequest><version>1.1</version>
                <xQuery xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true"/>
                <recordSchema>MARC21-xml</recordSchema></echoedSearchRetrieveRequest>
                </searchRetrieveResponse>''')

## 2. Datenset mit dem DNB SRU Query Tool erstellen

Das DNB SRU Query Tool kann hier heruntergeladen werden: [https://github.com/deutsche-nationalbibliothek/SRUQueryTool](https://github.com/deutsche-nationalbibliothek/SRUQueryTool/releases/tag/v1.0)

Nach dem Herunterladen einfach ausführen, Suchanfrage eingeben und Datenset erstellen lassen. 